本文学习于 [调参必备--Grid Search网格搜索](https://www.jianshu.com/p/55b9f2ea283b),[贝叶斯优化(Bayesian Optimization)深入理解](https://www.cnblogs.com/marsggbo/p/9866764.html)和 [scikit-learn User guide](http://sklearn.apachecn.org/#/docs/35),编辑:Weiyang,Time:2019.2.13,weixin:damo894127201 .

我们都知道机器学习模型是由许多超参数决定的，比如网络深度、学习率、卷积核大小等等，因此为了找到一个最好的超参数组合，最直观的想法就是**穷举搜索(即网格搜索)**。但是我们都知道机器学习训练模型是一个非常耗时的过程，如果想将每种可能的超参数组合都实验一遍，明显不现实，所以网格搜索一般事先限定若干种可能的参数组合，但是这样搜索仍然不高效。因此为了提高搜索效率，便提出了**随机搜索**。虽然随机搜索得到的结果互相之间差异较大，但实验证明**随机搜索的确比网格搜索效果更好**。

# GridSearch:网格搜索

## 概念

**网格搜索**又称为**穷举搜索**，在所有候选的每个参数列表中，通过循环遍历，尝试每一种可能性，表现最好的参数就是最终的结果。例如，有两个参数，参数α有三种可能，参数β有4种可能，把所有可能性都列出来后，可以表示为一个4\*3的表格，其中每格就是一个网格，循环过程就像是在每个网格里遍历搜索，故称为网格搜索。

|参数|$\alpha$=0.1|$\alpha$=0.01|$\alpha$=0.001|
|:--:|:---------:|:------------:|:------------:|
|$\beta$=0.1|model($\alpha$=0.1,$\beta$=0.1)|model($\alpha$=0.01,$\beta$=0.1)|model($\alpha$=0.001,$\beta$=0.1)|
|$\beta$=0.2|model($\alpha$=0.1,$\beta$=0.2)|model($\alpha$=0.01,$\beta$=0.2)|model($\alpha$=0.001,$\beta$=0.2)|
|$\beta$=0.3|model($\alpha$=0.1,$\beta$=0.3)|model($\alpha$=0.01,$\beta$=0.3)|model($\alpha$=0.001,$\beta$=0.3)|
|$\beta$=0.4|model($\alpha$=0.1,$\beta$=0.4)|model($\alpha$=0.01,$\beta$=0.4)|model($\alpha$=0.001,$\beta$=0.4)|

## 简单实现:以两个参数调优为例

### 构造分类伪数据

In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 构造数据集
X,Y = make_classification(n_samples=1000,
                          n_features=5,
                          n_informative=3,
                          n_redundant=1,
                          n_repeated=1,
                          n_classes=2,
                         )

# 划分训练集和测试集
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,shuffle=True)

### 设置待调优的两个参数选择范围

In [2]:
gammas = [0.001,0.01,0.1,1]
Cs = [0.002,0.03,0.4,1.2]

### 设置评估模型性能的指标

这里用SVC.score()函数，返回的是测试数据的平均准确率

In [3]:
best_score = 0

### 开始网格搜索

In [4]:
from sklearn.svm import SVC

raw_model = None
for gamma in gammas:
    for C in Cs:
        model = SVC(gamma=gamma,C=C) # 对于每种参数组合进行一次训练
        if raw_model == model: # 判断训练过程中，当前新生成的模型是否是前一训练轮次的模型
            print('前后两次训练使用的是同一模型')
        model.fit(X_train,Y_train)
        score = model.score(X_test,Y_test) # 模型的评估指标
        if score > best_score: # 寻找表现最好的参数
            best_score = score
            best_parameters = {'gamma':gamma,'C':C}
        raw_model = model
        
print('Best score:{:.2f}'.format(best_score))
print('Best parameters:{}'.format(best_parameters))

Best score:0.94
Best parameters:{'gamma': 1, 'C': 1.2}


虽然对于每种参数组合，我们都使用了纯净的模型来训练，表面上测试集和参数组合是独立的，但是事实上并非如此，每次训练时，训练集和测试集都是固定的，这就使得结果特别依赖于最初分裂的训练集和测试集，因此为得到更有说服力的**参数组合**，我们需要将**交叉验证**引入进来。

## 网格搜索+交叉验证：GridSearchCV

### 手动实现

#### 构造伪数据集

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

# 构造数据集
X,Y = make_classification(n_samples=1000,
                          n_features=5,
                          n_informative=3,
                          n_redundant=1,
                          n_repeated=1,
                          n_classes=2,
                         )
# 将原数据集转为numpy.array
X = np.array(X)
Y = np.array(Y)

#### 设置待调优的两个参数选择范围

In [6]:
gammas = [0.001,0.01,0.1,1]
Cs = [0.002,0.03,0.4,1.2]

#### 设置评估模型性能的指标

In [7]:
# 这里用SVC.score()函数，返回的是测试数据的平均准确率
best_score = 0

#### 网格搜索 + 交叉验证

In [8]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC


raw_model = None
# 交叉验证:10折交叉验证
kf = KFold(n_splits=10)
iters = 1
for train_index,test_index in kf.split(X):
    X_train,Y_train,X_test,Y_test = X[train_index],Y[train_index],X[test_index],Y[test_index]
    # 网格搜索
    for gamma in gammas:
        for C in Cs:
            model = SVC(gamma=gamma,C=C) # 对于每种参数组合进行一次训练
            if raw_model == model: # 判断训练过程中，当前新生成的模型是否是前一训练轮次的模型
                print('前后两次训练使用的是同一模型')
            model.fit(X_train,Y_train)
            score = model.score(X_test,Y_test) # 模型的评估指标
            if score > best_score: # 寻找表现最好的参数
                best_score = score
                best_parameters = {'gamma':gamma,'C':C}
            raw_model = model
    print('{}:iter,Best score:{:.2f},Best parameters:{}'.format(iters,best_score,best_parameters))
    iters += 1
    
print()  
print('Best score:{:.2f}'.format(best_score))
print('Best parameters:{}'.format(best_parameters))

1:iter,Best score:0.94,Best parameters:{'gamma': 0.1, 'C': 1.2}
2:iter,Best score:0.97,Best parameters:{'gamma': 0.1, 'C': 1.2}
3:iter,Best score:0.97,Best parameters:{'gamma': 0.1, 'C': 1.2}
4:iter,Best score:0.97,Best parameters:{'gamma': 0.1, 'C': 1.2}
5:iter,Best score:0.97,Best parameters:{'gamma': 0.1, 'C': 1.2}
6:iter,Best score:0.97,Best parameters:{'gamma': 0.1, 'C': 1.2}
7:iter,Best score:0.97,Best parameters:{'gamma': 0.1, 'C': 1.2}
8:iter,Best score:0.97,Best parameters:{'gamma': 0.1, 'C': 1.2}
9:iter,Best score:0.99,Best parameters:{'gamma': 1, 'C': 1.2}
10:iter,Best score:0.99,Best parameters:{'gamma': 1, 'C': 1.2}

Best score:0.99
Best parameters:{'gamma': 1, 'C': 1.2}


### sklearn实现: sklearn.model_selection.GridSearchCV

#### GridSearchCV 参数和类方法介绍

GridSearchCV() 将**网格搜索**和**交叉验证**结合起来，实现了fit()，predict(),score()等方法，重要的参数如下:
1. estimator:实现了scikit-learn的estimator接口，scikit-learn提供的模型都可以
2. param_grid:待优化的参数
3. cv:默认为None，代表3折交叉验证，可以取int，代表K折交叉验证

grid_search = GridSearchCV()
GridSearchCV()类实现的方法:
1. grid_search.fit(X,Y):用来训练模型和寻找最优参数组合，grid_search是那个最优参数组合的模型
2. grid_search.predict(X):在grid_search.fit(X,Y)之后使用，用最优参数组合的那个模型来预测，返回类别
3. grid_search.predict_log_proba(X):在grid_search.fit(X,Y)之后使用，用最优参数组合的那个模型来预测，返回取对数后的概率值
4. grid_search.predict_proba(X):在grid_search.fit(X,Y)之后使用，用最优参数组合的那个模型来预测，返回概率值
5. grid_search.score(X,Y):在grid_search.fit(X,Y)之后使用，用最优参数组合的那个模型来评估测试集，返回准确率

#### 构造伪数据

In [9]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

# 构造数据集
X,Y = make_classification(n_samples=1000,
                          n_features=5,
                          n_informative=3,
                          n_redundant=1,
                          n_repeated=1,
                          n_classes=2,
                         )
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

#### 待优化的参数

In [10]:
param_grid = {
    'gamma':[0.001,0.01,0.1,1],
    'C':[0.002,0.03,0.4,1.2]
}

#### 网格搜索+交叉验证

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# 实例化一个GridSearchCV类
grid_search = GridSearchCV(estimator=SVC(),param_grid=param_grid,cv=10)
# 将训练集进一步划分为训练集和验证集，训练模型,寻找最优参数组合
# 同时使用最优参数组合实例化的那个SVC estimator
grid_search.fit(X_train,Y_train)
# 用最优参数组合来评估测试集的性能指标得分
print("Test set score:{:.2f}".format(grid_search.score(X_test,Y_test)))
print("Best parameters:{}".format(grid_search.best_params_))
print("Best score on train set:{:.2f}".format(grid_search.best_score_))

Test set score:0.92
Best parameters:{'C': 1.2, 'gamma': 1}
Best score on train set:0.90


/home/weiyang/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# RandomSearch:随机搜索

## 概念

**核心思想**:从参数组合中随机抽取组合来优化模型。对于每个参数,可以指定在可能值上的分布或离散选择的列表，候选的每个参数值都是均匀取样获得的。对于连续参数，指定连续分布以充分利用随机化，这样有助于**n_iter(需要随机抽取的组合个数)**总是趋向于更精细的搜索。

## 使用随机搜索的前提

1. 数据规模大，精确的结果难以在一定时间计算出
2. 结果的些许的不精确能够被接受
3. 求取的结果是最优化（optimization）问题，有一个成本计算模型

## 随机搜索+交叉验证: RandomizedSearchCV

### RandomizedSearchCV 参数和类方法介绍

RandomizedSearchCV() 将**随机搜索**和**交叉验证**结合起来，实现了fit()，predict(),score()等方法，重要的参数如下:
1. estimator:实现了scikit-learn的estimator接口，scikit-learn提供的模型都可以
2. param_distributions:待优化的参数字典，如果是参数是离散取值，则值是列表形式；如果参数是连续分布的，则用scipy.stats模块表示分布，如{'gamma': scipy.stats.uniform(scale=.1)}表示参数gamma服从均匀分布
3. n_iter:表示随机抽取的参数组合个数
4. cv:默认为None，代表3折交叉验证，可以取int，代表K折交叉验证

random_search = RandomizedSearchCV()
RandomizedSearchCV()类实现的方法:
1. random_search.fit(X,Y):用来训练模型和寻找最优参数组合，random_search是那个最优参数组合的模型
2. random_search.predict(X):在random_search.fit(X,Y)之后使用，用最优参数组合的那个模型来预测，返回类别
3. random_search.predict_log_proba(X):在random_search.fit(X,Y)之后使用，用最优参数组合的那个模型来预测，返回取对数后的概率值
4. random_search.predict_proba(X):在random_search.fit(X,Y)之后使用，用最优参数组合的那个模型来预测，返回概率值
5. random_search.score(X,Y):在random_search.fit(X,Y)之后使用，用最优参数组合的那个模型来评估测试集，返回准确率

### 构造伪数据

In [12]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

# 构造数据集
X,Y = make_classification(n_samples=1000,
                          n_features=5,
                          n_informative=3,
                          n_redundant=1,
                          n_repeated=1,
                          n_classes=2,
                         )
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

### 待优化的参数

In [13]:
param_distributions = {
    'gamma':[0.001,0.01,0.1,1],
    'C':[0.002,0.03,0.4,1.2]
}

### 随机搜索+交叉验证

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

# 实例化一个RandomizedSearchCV类
random_search = RandomizedSearchCV(estimator=SVC(),
                                   param_distributions=param_distributions,
                                   n_iter = 10,
                                   cv=10)
# 将训练集进一步划分为训练集和验证集，训练模型,寻找最优参数组合
# 同时使用最优参数组合实例化的那个SVC estimator
random_search.fit(X_train,Y_train)
# 用最优参数组合来评估测试集的性能指标得分
print("Test set score:{:.2f}".format(random_search.score(X_test,Y_test)))
print("Best parameters:{}".format(random_search.best_params_))
print("Best score on train set:{:.2f}".format(random_search.best_score_))

Test set score:0.86
Best parameters:{'gamma': 1, 'C': 0.4}
Best score on train set:0.89
